In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

import gc

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./"))

# Any results you write to the current directory are saved as output.

['.DS_Store', 'test.csv', 'train_sample.csv', 'submission.csv', '.train_sample.csv.swp', '.gitignore', 'train.csv', '.ipynb_checkpoints', 'v1.ipynb', '.git']


In [2]:
path = './'

train_skip = range(1, 140000000)
train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8'
        }

In [3]:
print("loading training data")
train= pd.read_csv(path + "train.csv", skiprows=train_skip, dtype=dtypes, header=0, usecols=train_cols)
test = pd.read_csv(path + "test.csv", dtype=dtypes, header=0, usecols=test_cols)

loading training data


In [4]:
len_train = len(train)
train=train.append(test)
del test
gc.collect()

14

In [5]:
train['hour'] = pd.to_datetime(train.click_time).dt.hour.astype('uint8')
train['day'] = pd.to_datetime(train.click_time).dt.day.astype('uint8')

In [8]:
n_chans = train[['ip','day','hour','channel']].groupby(by=['ip','day',
          'hour'])[['channel']].count().reset_index().rename(columns={'channel': 'n_channels'})
print(n_chans)

             ip  day  hour  n_channels
0             0   10     4           2
1             1    9     3           6
2             1    9    10          11
3             1    9    12           2
4             1    9    13           1
5             1    9    14           6
6             1    9    15           2
7             1   10     4           1
8             2   10    10           1
9             2   10    13           4
10            3   10     4          10
11            3   10     5          27
12            3   10     9           6
13            3   10    13           2
14            3   10    14          19
15            4   10    14           6
16            5   10     5          21
17            5   10    10           2
18            5   10    13           2
19            6    9     2          14
20            6    9     3          26
21            6    9     4          11
22            6    9     5          55
23            6    9     6          38
24            6    9     

In [9]:
train = train.merge(n_chans, on=['ip', 'day', 'hour'], how='left')

In [10]:
train = train.drop(['click_time', 'ip'], axis=1)

In [11]:
train['n_channels'] = train['n_channels'].astype('uint16')

In [13]:
x = train.loc[:len_train - 1, train.columns != 'is_attributed']
y = train.loc[:len_train - 1, train.columns == 'is_attributed']

In [16]:
x_test = train.loc[len_train:, train.columns != 'is_attributed']

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=1)

In [ ]:
tree = DecisionTreeClassifier(max_depth=5)
model = AdaBoostClassifier(tree,
                         algorithm="SAMME",
                         n_estimators=200)

In [ ]:
model.fit(x_train, y_train)

/Users/acw/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
res = model.predict(x_test).astype(int)

In [26]:
submit = pd.read_csv(path + "test.csv", dtype='int', usecols=['click_id'])
submit['is_attributed'] = res

In [27]:
submit.to_csv('submission.csv', index=False)